In [ ]:
!git clone https://github.com/LuckyBoy587/Flappy-Bird-AI.git
!cd Flappy-Bird-AI

!pip install -q pygame==2.5.2 numpy==1.24.3 torch

In [ ]:
import os
os.listdir()

In [ ]:
from bird_ai import train_dqn
train_dqn()